# clamp -1,1

In [6]:
# from dianet import *
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

df = pd.read_csv('ionodata/iono.data', header=None)

# col[0:34] as X, col[34] as y
X = df.iloc[:, 0:34].values
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

y = df.iloc[:, 34].values
y = np.where(y == 'g', 1, 0)
print(X.shape, y.shape)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
y_test = torch.tensor(y_test, dtype=torch.long)

# define model, input dim = 34, output dim = 2
class baseline(nn.Module):
    def __init__(self):
        super(baseline, self).__init__()
        self.fc1 = nn.Linear(34, 16)
        self.fc2 = nn.Linear(16, 2)
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    
class dianet(nn.Module):
    def __init__(self):
        super(dianet, self).__init__()
        self.fc1 = nn.Linear(15, 9)
        self.fc2 = nn.Linear(15, 8)
        self.fc3 = nn.Linear(13, 7)
        self.fc4 = nn.Linear(11, 6)
        self.fc5 = nn.Linear(9, 5)
        self.fc6 = nn.Linear(6, 4)
        self.fc7 = nn.Linear(4, 3)
        self.fc8 = nn.Linear(3, 2)
        self.msk1 = np.array([[1,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[1,1,1,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,1,1,1,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,1,1,1,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,1,1,1,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,1,1,1,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,1,1,1,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,1,1,1],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,1]])
        self.msk2 = np.array([[1,1,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,1,1,1,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,1,1,1,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,1,1,1,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,1,1,1,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,1,1,1,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,1,1,1,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,1,1]])
        self.msk3 = np.array([[1,1,0,0,0,0,0,0,0,0,0,0,0],[0,1,1,1,0,0,0,0,0,0,0,0,0],[0,0,0,1,1,1,0,0,0,0,0,0,0],[0,0,0,0,0,1,1,1,0,0,0,0,0],[0,0,0,0,0,0,0,1,1,1,0,0,0],[0,0,0,0,0,0,0,0,0,1,1,1,0],[0,0,0,0,0,0,0,0,0,0,0,1,1]])
        self.msk4 = np.array([[1,1,0,0,0,0,0,0,0,0,0],[0,1,1,1,0,0,0,0,0,0,0],[0,0,0,1,1,1,0,0,0,0,0],[0,0,0,0,0,1,1,1,0,0,0],[0,0,0,0,0,0,0,1,1,1,0],[0,0,0,0,0,0,0,0,0,1,1]])
        self.msk5 = np.array([[1,1,0,0,0,0,0,0,0],[0,1,1,1,0,0,0,0,0],[0,0,0,1,1,1,0,0,0],[0,0,0,0,0,1,1,1,0],[0,0,0,0,0,0,0,1,1]])
        self.msk6 = np.array([[1,1,0,0,0,0],[0,1,1,1,0,0],[0,0,0,1,1,0],[0,0,0,0,1,1]])
        self.msk7 = np.array([[1,1,0,0],[0,1,1,0],[0,0,1,1]])
        self.msk8 = np.array([[1,1,0],[0,1,1]])
    def forward(self, x, dwn, up):
        self.fc1.weight.data *= torch.from_numpy(self.msk1).float()
        self.fc1.weight.data = torch.clamp(self.fc1.weight.data, min=dwn, max=up)
        self.fc1.bias.data = torch.clamp(self.fc1.bias.data, min=dwn, max=up)
        x1 = torch.tanh(self.fc1(x[:,0:15])) # 9
        x1c = torch.cat((x1[:,0:2],x[:,15:16],x1[:,2:3],x[:,16:17],x1[:,3:4],x[:,17:18],x1[:,4:5],x[:,18:19],x1[:,5:6],x[:,19:20],x1[:,6:7],x[:,20:21],x1[:,7:9]),1) # 15
        
        self.fc2.weight.data *= torch.from_numpy(self.msk2).float()
        self.fc2.weight.data = torch.clamp(self.fc2.weight.data, min=dwn, max=up)
        self.fc2.bias.data = torch.clamp(self.fc2.bias.data, min=dwn, max=up)
        x2 = torch.tanh(self.fc2(x1c)) # 8
        x2c = torch.cat((x2[:,0:2],x[:,21:22],x2[:,2:3],x[:,22:23],x2[:,3:4],x[:,23:24],x2[:,4:5],x[:,24:25],x2[:,5:6],x[:,25:26],x2[:,6:8]),1) # 13
        
        self.fc3.weight.data *= torch.from_numpy(self.msk3).float()
        self.fc3.weight.data = torch.clamp(self.fc3.weight.data, min=dwn, max=up)
        self.fc3.bias.data = torch.clamp(self.fc3.bias.data, min=dwn, max=up)
        x3 = torch.tanh(self.fc3(x2c)) # 7
        x3 = x3+x1[:,1:-1] # 7
        x3c = torch.cat((x3[:,0:2],x[:,26:27],x3[:,2:3],x[:,27:28],x3[:,3:4],x[:,28:29],x3[:,4:5],x[:,29:30],x3[:,5:7]),1) # 11
        
        self.fc4.weight.data *= torch.from_numpy(self.msk4).float()
        self.fc4.weight.data = torch.clamp(self.fc4.weight.data, min=dwn, max=up)
        self.fc4.bias.data = torch.clamp(self.fc4.bias.data, min=dwn, max=up)
        x4 = torch.tanh(self.fc4(x3c)) # 6
        x4 = x4+x2[:,1:-1] # 6
        x4c = torch.cat((x4[:,0:2],x[:,30:31],x4[:,2:3],x[:,31:32],x4[:,3:4],x[:,32:33],x4[:,4:6]),1) # 9

        self.fc5.weight.data *= torch.from_numpy(self.msk5).float()
        self.fc5.weight.data = torch.clamp(self.fc5.weight.data, min=dwn, max=up)
        self.fc5.bias.data = torch.clamp(self.fc5.bias.data, min=dwn, max=up)
        x5 = torch.tanh(self.fc5(x4c)) # 5
        x5 = x5+x3[:,1:-1] # 5
        x5c = torch.cat((x5[:,0:2],x[:,33:34],x5[:,2:5]),1) # 6

        self.fc6.weight.data *= torch.from_numpy(self.msk6).float()
        self.fc6.weight.data = torch.clamp(self.fc6.weight.data, min=dwn, max=up)
        self.fc6.bias.data = torch.clamp(self.fc6.bias.data, min=dwn, max=up)
        x6 = torch.tanh(self.fc6(x5c)) # 4
        x6 = x6+x4[:,1:-1] # 4

        self.fc7.weight.data *= torch.from_numpy(self.msk7).float()
        self.fc7.weight.data = torch.clamp(self.fc7.weight.data, min=dwn, max=up)
        self.fc7.bias.data = torch.clamp(self.fc7.bias.data, min=dwn, max=up)
        x7 = torch.tanh(self.fc7(x6)) # 3
        x7 = x7+x5[:,1:-1] # 3

        self.fc8.weight.data *= torch.from_numpy(self.msk8).float()
        self.fc8.weight.data = torch.clamp(self.fc8.weight.data, min=dwn, max=up)
        self.fc8.bias.data = torch.clamp(self.fc8.bias.data, min=dwn, max=up)
        x8 = torch.tanh(self.fc8(x7)) # 2
        x8 = x8+x6[:,1:-1] # 2
        return x8



(351, 34) (351,)


In [8]:
from sklearn.metrics import classification_report
model = dianet()
# model = baseline()
# model.load_state_dict(torch.load('ionodata/baseline.pth'))
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

min_loss = 99
maxACC = 0.01
for epoch in range(1,301):
    batch_size = 4
    epoch_loss = 0
    count = 0
    for i in range(0, X_train.shape[0], batch_size):
        X_batch = X_train[i:i+batch_size]
        y_batch = y_train[i:i+batch_size]
        y_pred = model(X_batch, -1, 1)
        loss = loss_fn(y_pred, y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        count += 1
    epoch_loss /= count
    # if epoch_loss < min_loss:
    #     min_loss = epoch_loss
    #     torch.save(model.state_dict(), 'ionodata/baseline.pth')
    #     print('saved at epoch: ', epoch, 'loss: ', epoch_loss)

    # test current model
    y_pred = model(X_test, -1, 1)
    y_pred = np.argmax(y_pred.detach().numpy(), axis=1)
    if classification_report(y_test, y_pred, output_dict=True)['weighted avg']['f1-score'] > maxACC:
        maxACC = classification_report(y_test, y_pred, output_dict=True)['weighted avg']['f1-score']
        torch.save(model.state_dict(), 'ionodata/dianet-11.pth')
        print('saved at epoch: ', epoch, ' loss: ', epoch_loss, ' ACC: ', maxACC)

c:\Users\comparch\anaconda3\envs\CondaPy39\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\comparch\anaconda3\envs\CondaPy39\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\comparch\anaconda3\envs\CondaPy39\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c

saved at epoch:  1  loss:  0.6560162200203424  ACC:  0.4298245614035088


c:\Users\comparch\anaconda3\envs\CondaPy39\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\comparch\anaconda3\envs\CondaPy39\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\comparch\anaconda3\envs\CondaPy39\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c

saved at epoch:  9  loss:  0.5915851513796215  ACC:  0.4895833333333333
saved at epoch:  11  loss:  0.5490613821186597  ACC:  0.5925925925925926
saved at epoch:  12  loss:  0.5204296481760242  ACC:  0.7551515151515152
saved at epoch:  13  loss:  0.49122406070745445  ACC:  0.7898550724637682
saved at epoch:  23  loss:  0.3434848609981658  ACC:  0.8229166666666666
saved at epoch:  104  loss:  0.22378768307404429  ACC:  0.8546099290780141
saved at epoch:  282  loss:  0.1835956471158734  ACC:  0.8851727982162765


In [10]:
# load model
model.load_state_dict(torch.load('ionodata/dianet-11.pth'))
y_pred = model(X_test, -1, 1)
y_pred = np.argmax(y_pred.detach().numpy(), axis=1)
print(classification_report(y_test, y_pred))
# print(classification_report(y_test, y_pred, output_dict=True)['weighted avg']['f1-score'])

              precision    recall  f1-score   support

           0       1.00      0.73      0.85        15
           1       0.84      1.00      0.91        21

    accuracy                           0.89        36
   macro avg       0.92      0.87      0.88        36
weighted avg       0.91      0.89      0.89        36



In [11]:
# print model.fc1.weignt.data
print(model.fc1.weight.data)
# print model.fc1.bias.data
print(model.fc1.bias.data)

tensor([[ 0.5065, -0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.9999, -0.1162,  0.3380,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [-0.0000,  0.0000, -0.7385, -0.9994, -0.1627, -0.0000, -0.0000, -0.0000,
         -0.0000, -0.0000, -0.0000, -0.0000, -0.0000, -0.0000, -0.0000],
        [-0.0000, -0.0000, -0.0000, -0.0000, -0.6570, -0.4124, -0.1054, -0.0000,
         -0.0000, -0.0000, -0.0000, -0.0000, -0.0000, -0.0000, -0.0000],
        [-0.0000, -0.0000, -0.0000, -0.0000, -0.0000, -0.0000, -0.2165, -0.7418,
         -0.3563, -0.0000, -0.0000, -0.0000, -0.0000, -0.0000, -0.0000],
        [ 0.0000, -0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.2925, -0.1380, -0.2968,  0.0000,  0.0000,  0.0000,  0.0000],
        [-0.0000, -0.0000, -0.0000, -0.0000, -0.0000, -0.0000, -0.0000, -0.0

# clamp 0,1

In [12]:
from sklearn.metrics import classification_report
model = dianet()
# model = baseline()
# model.load_state_dict(torch.load('ionodata/baseline.pth'))
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

min_loss = 99
maxACC = 0.01
for epoch in range(1,301):
    batch_size = 4
    epoch_loss = 0
    count = 0
    for i in range(0, X_train.shape[0], batch_size):
        X_batch = X_train[i:i+batch_size]
        y_batch = y_train[i:i+batch_size]
        y_pred = model(X_batch, 0, 1)
        loss = loss_fn(y_pred, y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        count += 1
    epoch_loss /= count

    # test current model
    y_pred = model(X_test, 0, 1)
    y_pred = np.argmax(y_pred.detach().numpy(), axis=1)
    if classification_report(y_test, y_pred, output_dict=True)['weighted avg']['f1-score'] > maxACC:
        maxACC = classification_report(y_test, y_pred, output_dict=True)['weighted avg']['f1-score']
        torch.save(model.state_dict(), 'ionodata/dianet01.pth')
        print('saved at epoch: ', epoch, ' loss: ', epoch_loss, ' ACC: ', maxACC)

c:\Users\comparch\anaconda3\envs\CondaPy39\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\comparch\anaconda3\envs\CondaPy39\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\comparch\anaconda3\envs\CondaPy39\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c

saved at epoch:  1  loss:  0.7038485573816903  ACC:  0.4298245614035088


c:\Users\comparch\anaconda3\envs\CondaPy39\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\comparch\anaconda3\envs\CondaPy39\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\comparch\anaconda3\envs\CondaPy39\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c

saved at epoch:  20  loss:  0.6032655789882322  ACC:  0.4895833333333333
saved at epoch:  45  loss:  0.572807905794699  ACC:  0.54349376114082


In [13]:
# load model
model.load_state_dict(torch.load('ionodata/dianet01.pth'))
y_pred = model(X_test, -1, 1)
y_pred = np.argmax(y_pred.detach().numpy(), axis=1)
print(classification_report(y_test, y_pred))
# print(classification_report(y_test, y_pred, output_dict=True)['weighted avg']['f1-score'])

              precision    recall  f1-score   support

           0       1.00      0.13      0.24        15
           1       0.62      1.00      0.76        21

    accuracy                           0.64        36
   macro avg       0.81      0.57      0.50        36
weighted avg       0.78      0.64      0.54        36

